<a href="https://colab.research.google.com/github/akariiijima/GAN/blob/master/practiceGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Reshape, Conv2D, Input
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D, Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Flatten, Dropout
import math
import numpy as np
import os
from keras.datasets import mnist
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt

In [0]:
class MnistGAN:
  
  def __init__(self):
    self.z_dim = 100
    self.shape = (28,28,1)
    optimizer = Adam(lr=1e-4, beta_1=0.5)
    
    # build discriminator
    self.discriminator = self.build_discriminator()
    self.discriminator.compile(loss='binary_crossentropy', \
                               optimizer=optimizer, \
                               metrics=['acc'])
    
    # build generator 
    self.generator = self.build_generator()   
    z = Input(shape=(self.z_dim,))
    img = self.generator(z)
    
    # fix the parameters of discriminator
    self.discriminator.trainable = False
    valid = self.discriminator(img)
    
    # for training generator
    self.combined = Model(z, valid)
    self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
  
  
  def build_generator(self):
    noise_shape = (self.z_dim,)    
    model = Sequential()
    
    #-----------------------#
    model.add(Dense(7*7*128, activation='relu', input_shape=noise_shape))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())                   # (14,14,128)
    #-----------------------#
    model.add(Conv2D(8, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())                   # (28,28,128)
    #-----------------------#
    model.add(Conv2D(4, kernel_size=3, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(momentum=0.8)) # (28,28,64) 
    #-----------------------#
    model.add(Conv2D(1, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    
    model.summary()
    
    noise = Input(shape=noise_shape)
    
    img = model(noise)
    
    return Model(inputs=noise, outputs=img)
  
  def build_discriminator(self):
    img_shape = self.shape
    
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape,\
                    padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    #--------------------------------
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    #--------------------------------
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    
    img = Input(shape=img_shape)
    validity = model(img)
    
    return Model(inputs=img, outputs=validity)
  
  def train(self, iterations, batch_size=128, save_interval=50, \
            model_interval=1000, check_noise=None, r=5, c=5):
    x_train, y_train, x_test, y_test = self.load_imgs()
    
    half_batch = int(batch_size / 2)
    
    x_train = (x_train.astype(np.float32) - 127.5) / 127.5
    
    for iteration in range(iterations):
      """
      Training Discriminator
      """
      idx = np.random.randint(0, x_train.shape[0], half_batch)
      
      imgs = x_train[idx]
      
      noise = np.random.uniform(-1, 1, (half_batch, self.z_dim))
      gen_imgs = self.generator.predict(noise)
      
      
      d_loss_real = self.discriminator.train_on_batch(imgs, \
                                                      np.ones((half_batch, 1)))
      d_loss_fake = self.discriminator.train_on_batch(gen_imgs, \
                                                      np.zeros((half_batch, 1)))
    
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      
      """
      Training Generator
      """
      noise = np.random.uniform(-1, 1, (batch_size, self.z_dim))
      
      g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))
      
      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration, d_loss[0], 100 * d_loss[1], g_loss))
      for i in range(9):
        plotImg = gen_imgs[i].reshape(28,28)
        plt.subplot(3,3,i+1)
        plt.imshow(plotImg, cmap='gray')
      plt.show()
      
    
  def load_imgs(self):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
    x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],x_test.shape[2],1))
    return x_train, y_train, x_test, y_test

In [0]:
mnistGAN = MnistGAN()
mnistGAN.train(iterations=3000)